In [40]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

df = sqlc.read.csv('file:///home/oxclo/datafiles/practices/ukpractices2015.csv', header='true', inferSchema='true')
df.show(5)

+------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  date|    id|                name|            address1|            address2|                town|            district|            postcode|               extra|
+------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|201512|A81001|THE DENSHAM SURGE...|THE HEALTH CENTRE...|LAWSON STREET    ...|STOCKTON ON TEES ...|CLEVELAND        ...|TS18 1HU         ...|                 ...|
|201512|A81002|QUEENS PARK MEDIC...|QUEENS PARK MEDIC...|FARRER STREET    ...|STOCKTON ON TEES ...|CLEVELAND        ...|TS18 2AW         ...|                 ...|
|201512|A81003|VICTORIA MEDICAL ...|THE HEALTH CENTRE...|VICTORIA ROAD    ...|HARTLEPOOL       ...|CLEVELAND        ...|TS26 8DB         ...|                 ...|
|201512|A81004|WOODLAN

In [56]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import split
sqlc = SQLContext(sc)

df = sqlc.read.csv('file:///home/oxclo/datafiles/practices/ukpractices2015.csv', header='true', inferSchema='true')
# df.show(5)

split_col = split(df['postcode'], ' ')
df = df.withColumn('post', split_col.getItem(0))

# Use Functional approach
mapped = df.rdd.map(lambda x: (x.post, 1)).filter(lambda x: 'OX1' in x or 'SW11' in x)
# reduce style
count = mapped.reduceByKey(lambda a,b: a+b)
sortedCount = count.sortBy(lambda (a,b): b, ascending=False)
for k,v in sortedCount.collect(): print k,v
## countByKey style - note it returns a python object  
## count = mapped.countByKey().items()
## for k,v in count: print k,v



SW11 15
OX1 7


In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import split

sqlc = SQLContext(sc)

df = sqlc.read.csv('file:///home/oxclo/datafiles/practices/ukpractices2015.csv', header='true', inferSchema='true')
# df.show(5)

split_col = split(df['postcode'], ' ')
df = df.withColumn('post', split_col.getItem(0))

# Use SQL approach
df.registerTempTable('practices')
sqlc.sql("Select post, count(post) as sum from practices where post in ('OX1', 'SW11') group by post order by post desc").show()

+----+---+
|post|sum|
+----+---+
|SW11| 15|
| OX1|  7|
+----+---+



In [15]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import split

sqlc = SQLContext(sc)

df = sqlc.read.csv('file:///home/oxclo/datafiles/practices/ukpractices2015.csv', header='true', inferSchema='true')
# df.show(5)

sqlc.registerFunction('f', lambda p: p.split()[0])
                                              
# split_col = split(df['postcode'], ' ')
# df = df.withColumn('post', split_col.getItem(0))

# Use SQL approach
df.registerTempTable('practices')
sqlc.sql("Select f(postcode) as pc, sum(1) from practices where f(postcode) in ('OX1', 'SW11') group by pc order by pc desc").show()

+----+------+
|  pc|sum(1)|
+----+------+
|SW11|    15|
| OX1|     7|
+----+------+

